In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
##os.chdir('/content/drive/MyDrive/PROJ_ERIC/')
os.chdir('/content/drive/MyDrive/Colab Notebooks/PROJ_ERIC/')
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
%pip install -U segmentation-models
%pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
# import libs
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import random
import time
from utils.tools import *
from core import postprocess
import pydensecrf
import cv2
print(tf.__version__)

In [ ]:
def return_zeroed(mask, crf_mask):
    ny, nx, nz = mask.shape
    for z in range(nz):
        summ = np.sum(mask[:, :, z])
        if summ == 0:
            crf_mask[:, :, z] = 0.
    return crf_mask

In [ ]:
def dense_crf(image, final_probabilities, gw=11, bw=3, n_iterations=5):

    import pydensecrf.densecrf as dcrf
    from pydensecrf.utils import create_pairwise_gaussian, create_pairwise_bilateral, unary_from_softmax

    '''
    gw - pairwise gaussian window size: enforces more spatially consistent segmentations.
    bw - pairwise bilateral window size: uses local color features to refine predictions.
    '''

    ny = image.shape[0]
    nx = image.shape[1]
    n_classes = final_probabilities.shape[-1]
    softmax = final_probabilities.squeeze()
    softmax = softmax.transpose((2, 0, 1))

	# The input should be the negative of the logarithm of probability values
	# Look up the definition of the unary_from_softmax for more information
    unary = unary_from_softmax(softmax, scale=None, clip=1e-5)

	# The inputs should be C-continious -- we are using Cython wrapper
    unary = np.ascontiguousarray(unary)

    d = dcrf.DenseCRF(ny*nx, n_classes)

    d.setUnaryEnergy(unary)

	# This potential penalizes small pieces of segmentation that are
	# spatially isolated -- enforces more spatially consistent segmentations
    feats = create_pairwise_gaussian(sdims=(gw, gw), shape=(ny, nx))

    d.addPairwiseEnergy(feats, compat=3,
                    	kernel=dcrf.DIAG_KERNEL,
                    	normalization=dcrf.NORMALIZE_SYMMETRIC)

	# This creates the color-dependent features --
	# because the segmentation that we get from CNN are too coarse
	# and we can use local color features to refine them
    feats = create_pairwise_bilateral(sdims=(bw, bw), schan=(7, 7, 7),
									img=image, chdim=2)

    d.addPairwiseEnergy(feats, compat=3,
                     	kernel=dcrf.DIAG_KERNEL,
                     	normalization=dcrf.NORMALIZE_SYMMETRIC)

    Q = d.inference(n_iterations)
    probs = np.array(Q, dtype=np.float32).reshape((n_classes, ny, nx))
    probs = np.around(probs, 4)
    #res = np.argmax(Q, axis=0).reshape((ny, nx))

    return probs.swapaxes(1, 0).swapaxes(1, 2)

In [ ]:
# check the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import pickle
with open('dataset/dataset_forages_128x128_20230705.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
X_train, Y_train, y_train =  dataset['X_train'], dataset['Y_train'], dataset['y_train']
X_test, Y_test, y_test =  dataset['X_test'], dataset['Y_test'], dataset['y_test']
classes = Y_train.shape[-1]
print(X_train.shape)

In [ ]:
counts = np.unique(y_train, return_counts=True)[1]
n_samples = np.min(counts)
replace = False

indexes = []
for ii in range(classes):
    class_idx = np.where(y_train == ii)[0]
    indexes.append(np.random.choice(class_idx, size=n_samples, replace=replace))
indexes = np.concatenate(indexes)

X_train, Y_train = X_train[indexes], Y_train[indexes]

In [ ]:
def preprocess_batches(X, Y, fill_with_local_mean=False, pred_model=True):
    from scipy.stats import mode

    n = 0
    for im_i, m_i in tqdm(zip(X, Y)):


        fill_mean = np.mean(mode(im_i, keepdims=True)[0])
        idy, idx, _ = np.where(im_i != fill_mean)
        local_mean = np.mean(im_i[idy, idx])
        iy, ix, _ = np.where(im_i == fill_mean)
        if fill_with_local_mean:
            im_i = np.where(im_i == fill_mean, local_mean, im_i)
        else:
            im_i = np.where(im_i == fill_mean, 0., im_i)

        m_i[iy, ix] = 0.

        bilat_img = np.float32(cv2.bilateralFilter(np.float32(im_i), d=3, sigmaColor=15, sigmaSpace=25))
        if np.isnan(bilat_img).any():
            bilat_img = np.nan_to_num(bilat_img, nan=np.nanmean(bilat_img))

        crf_mask = dense_crf(im_i, m_i, gw=5, bw=7, n_iterations=1)
        crf_mask[iy, ix] = 0.
        X[n] = bilat_img
        Y[n] = return_zeroed(m_i, crf_mask)
        n += 1

    return X, Y

In [ ]:
batch_size = 500
for i in range(0, X_train.shape[0], batch_size):
  print(i)
  X_train[i:i+batch_size], Y_train[i:i+batch_size] = preprocess_batches(X_train[i:i+batch_size], Y_train[i:i+batch_size])

In [ ]:
augdata = True

if augdata:

    augdata = data_augmentation(X_train, Y_train)
    X_train, Y_train = augdata.rotation(nrot=[0, 2], perc=1.)

print(X_train.shape)

### Visualize

In [ ]:
for _ in range(5):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 4))
    ii = np.random.choice(np.arange(0, X_train.shape[0], 1, dtype=int))
    ax1.imshow(adjust_rgb(X_train[ii], 2, 98))
    ax2.imshow(Y_train[ii, :, :, 0])
    ax3.imshow(Y_train[ii, :, :, 1])
    ax4.imshow(Y_train[ii, :, :, 2])
    ax1.axis('off'); ax2.axis('off'); ax3.axis('off'); ax4.axis('off')
    plt.show()

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from sklearn.metrics import confusion_matrix
from keras import backend as K
from keras import callbacks
import segmentation_models as sm

In [ ]:
dim = X_train.shape[1:]
proportions = np.sum(Y_train.reshape(Y_train.shape[0]*Y_train.shape[1]*Y_train.shape[2], Y_train.shape[-1]), 0)
total_pixels = Y_train.size
wmatrix = np.array(np.kron(total_pixels/proportions, np.ones((1, dim[0], dim[1], 1))), dtype=np.float32)
wmatrix = 0.

In [ ]:
class masked_loss:

    def __init__(self, dim, ths, wmatrix=0, use_weights=False, hold_out=0.1):
        self.dim = dim
        self.ths = ths
        self.wmatrix = wmatrix
        self.use_weights = use_weights
        self.hold_out = hold_out

    def masked_rmse(self, y_true, y_pred):

        # distance between the predictions and simulation probabilities
        squared_diff = K.square(y_true-y_pred)

        # gives different weights by class
        if self.use_weights:
            squared_diff *= self.wmatrix

        # calculate the loss only where there are samples
        mask = tf.where(y_true >= self.ths, 1.0, 0.)

        # take some of the training points out at random
        if self.hold_out > 0:
            mask *= tf.where(tf.random.uniform(shape=(1, *squared_diff.shape[1:]), minval=0., maxval=1.) > self.hold_out, 1.0, 0.)

        denominator = K.sum(mask) # number of pixels
        if self.use_weights:
            denominator = K.sum(mask*self.wmatrix)

        # sum of squared differences at sampled locations
        summ = K.sum(squared_diff*mask)
        # calculate error
        rmse = K.sqrt(summ/denominator)


        return rmse

    def dice_loss(self, y_true, y_pred):

        # Dice coefficient loss
        y_pred = tf.cast(y_pred > 0.5, dtype=tf.float32)
        intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
        union = K.sum(y_true + y_pred, axis=[1, 2, 3]) - intersection
        dice_loss = 1. - (2. * intersection + 1.) / (union + 1.)

        return dice_loss


    def contrastive_loss(self, y_true, y_pred):

        # compute loss
        rmse = self.masked_rmse(y_true, y_pred)
        dice_loss = self.dice_loss(y_true, y_pred)

        loss = rmse + dice_loss

        return loss

### Defining the model

In [ ]:
BACKBONE = 'efficientnetb7'
preprocess_input = sm.get_preprocessing(BACKBONE)

# preprocess input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [ ]:
load_weights = False
loss = masked_loss(dim, ths=0.5, hold_out=0.1)

if load_weights is False:
    # define model
    model = sm.Linknet(BACKBONE, classes=classes, activation='softmax', encoder_weights='imagenet', encoder_freeze=False)

    # learning ratio
    lr = 1.5e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer,
        loss=loss.contrastive_loss,
        metrics=['acc'],
    )
#model.summary()

In [ ]:
from datetime import date
# get today's date
today = str(date.today()).replace('-', '_')

checkpoint_filepath = "models/save_models/"
sub_path = f'linket_{BACKBONE}_weights_{today}.h5'
checkpointer = callbacks.ModelCheckpoint(filepath=checkpoint_filepath+sub_path,
                                         monitor='loss', verbose=1, save_best_only=True, mode='min')

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=10e-4, patience=50)
callbacks_list = [checkpointer, early_stopping]

In [ ]:
from keras.models import load_model
if load_weights:
    saved_path = f'linknet_efficientnetb7_weights_2023_07_05.h5'
    lr = 1.5e-5
    model = tf.keras.models.load_model(checkpoint_filepath+saved_path, compile=False)
    loss = masked_loss(dim, ths=0.5, hold_out=0.1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer,
        loss=loss.contrastive_loss,
        metrics=['acc'],
    )

In [ ]:
training = True
batch_size = 16
if training ==True :
    history = model.fit(X_train,
                        Y_train,
                        batch_size=batch_size,
                        validation_data=(X_test, Y_test),
                        callbacks=[callbacks_list],
                        epochs=250)

In [ ]:
if training:
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(f'plots/graph_losses_{today}.png', dpi=300, bbox_inches='tight')
    plt.show()

### Visualize predictions

In [ ]:
b, e = 600, 610
pred_probs = model.predict(X_train[b:e])

n = 0
for i in range(b, e):

    fig, ax = plt.subplots(1, 4, figsize=(15, 6))

    ax[0].imshow(adjust_rgb(X_train[i], 10, 90))
    ax[1].imshow(pred_probs[n, :, :, 0], cmap='plasma', vmin=0, vmax=1)
    ax[2].imshow(pred_probs[n, :, :, 1], cmap='plasma', vmin=0, vmax=1)
    ax[3].imshow(pred_probs[n, :, :, 2], cmap='plasma', vmin=0, vmax=1)
    ax[0].axis('off'); ax[1].axis('off'); ax[2].axis('off'); ax[3].axis('off');
    n += 1

In [ ]:
b,e = 60, 70
pred_probs = model.predict(X_test[b:e])

n = 0
for i in range(b, e):

    fig, ax = plt.subplots(1, 5, figsize=(15, 6))

    ax[0].imshow(adjust_rgb(X_test[i], 10, 90))
    ax[1].imshow(Y_test[i, :, :, 1], cmap='plasma', vmin=0, vmax=1)
    ax[2].imshow(pred_probs[n, :, :, 0], cmap='plasma', vmin=0, vmax=1)
    ax[3].imshow(pred_probs[n, :, :, 1], cmap='plasma', vmin=0, vmax=1)
    ax[4].imshow(pred_probs[n, :, :, 2], cmap='plasma', vmin=0, vmax=1)
    ax[0].axis('off'); ax[1].axis('off'); ax[2].axis('off'); ax[3].axis('off'); ax[4].axis('off')
    n += 1

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

folder_path = 'images'
image_list = []

# Parcours de tous les fichiers du dossier
for filename in os.listdir(folder_path):
    if filename.endswith('.JPG') or filename.endswith('.jpeg'):
        # Chargement de l'image et ajout à la liste
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        image_list.append(np.array(img))
        plt.imshow(img)
        plt.show()

In [ ]:
def undersample(image, mask=False, undersample_by=1):
    yy = np.arange(0, image.shape[0], undersample_by)
    xx = np.arange(0, image.shape[1], undersample_by)

    idx, idy = np.meshgrid(xx, yy)

    ny = idy.shape[0]
    nx = idy.shape[1]

    resampled_image = image[idy.ravel(), idx.ravel(), :].reshape((ny, nx, 3))
    if mask:
       resampled_mask = mask[idy.ravel(), idx.ravel(), :].reshape((ny, nx, mask.shape[-1]))

    return resampled_image

## Applying the model to the set of images


In [ ]:
ii = np.random.choice(len(image_list), size=1)[0]
image = undersample(image_list[ii], undersample_by=1)
dim    = X_train.shape[1:]
XX = np.float32(cv2.bilateralFilter(np.float32(image), d=5, sigmaColor=35, sigmaSpace=35))
XX = preprocess_input(XX)
median_pixel_value = np.median(image[:100, :100])
imy, imx = np.where(image == median_pixel_value)[:2]
XX[imy, imx] = 0.

In [ ]:
pred_tile = postprocess.predict_tiles(model, merge_func=np.max, reflect=True)
pred_tile.create_batches(XX, (dim[0], dim[1], 3), step=int(dim[0]), n_classes=classes)
pred_tile.predict(batches_num=1500, coords_channels=False)
result = pred_tile.merge()
result[imy, imx] = 0.

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
ax.imshow(adjust_rgb(XX, 5, 99))
ax.imshow(result[:, :, 1],  cmap='viridis', vmin=0., vmax=1., alpha=0.)
plt.show()

In [ ]:
y = np.arange(result.shape[0])
x = np.arange(result.shape[1])
x, y = np.meshgrid(x, y)

In [ ]:
for c in range(classes):
  fig, ax = plt.subplots(figsize=(15, 15))
  ax.imshow(adjust_rgb(XX, 5, 99), zorder=0)
  ax.pcolormesh(x, y, np.where(result[:, :, c] > 0.9, 1., np.nan),  cmap='plasma', vmin=0.3, vmax=1., alpha=0.7, zorder=1)
  plt.xlim(70, 2300)
  plt.ylim(200, 1800)
  plt.axis('off')
  plt.savefig(f'plots/pred_{c}.png', dpi=300, bbox_inches='tight')
  plt.show()